# <font color = 'indianred'>**Decoder_Instruction_LMHead_Binary** </font>























# <font color = 'indianred'> **Setting up the Environment** </font>



In [ ]:
!nvidia-smi

Mon Nov  3 02:22:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   32C    P0             55W /  400W |       5MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  !pip install numpy -U -qq
  !pip install transformers evaluate wandb datasets accelerate trl peft bitsandbytes -U -qq
  !pip uninstall tensorflow

 <Font size = 5 color = 'indianred'>**Restart the session before moving onto next cell**
> Runtime- Restart Session

<font color = 'indianred'> *Load Libraries* </font>

In [ ]:
# standard python libraries
from pathlib import Path
from typing import Dict, List, Union, Optional, Tuple
from tqdm import tqdm
import json
import joblib
import os
import sys

import numpy as np
import seaborn as sns
from pathlib import Path
import re
import gc
import time

# Data Science librraies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Pytorch
import torch
import torch.nn as nn

# Huggingface Librraies
import evaluate
from datasets import load_dataset, DatasetDict, Dataset, ClassLabel
from trl import SFTConfig, SFTTrainer
from transformers import (
    set_seed,
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    AutoConfig,
    BitsAndBytesConfig,
)
from peft import (
    TaskType,
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    AutoPeftModelForCausalLM,
    PeftConfig
)
# Logging and secrets
from huggingface_hub import login, HfApi, create_repo
from google.colab import userdata
import wandb



In [ ]:
# Determine the storage location based on the execution environment
# If running on Google Colab, use Google Drive as storage
# CHANGE FOLDERS TO WHERE YOU WANT TO SAVE DATA AND MODELS

if 'google.colab' in str(get_ipython()):
    from google.colab import drive  # Import Google Drive mounting utility
    drive.mount('/content/drive')  # Mount Google Drive

    # Set base folder path for storing data on Google Drive
    base_folder= Path('/content/drive/MyDrive/data')

# If running locally, specify a different path
else:
    # Set base folder path for storing data on local machine
    base_folder= Path('/Users/ronchaudhuri/Documents/Models/Data')

Mounted at /content/drive


In [ ]:
wandb_api_key = userdata.get('WANDB_API_KEY')
hf_token = userdata.get('HF_Token')

In [ ]:
if hf_token:
    # Log in to Hugging Face
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Hugging Face token not found in notebook secrets.")

Successfully logged in to Hugging Face!


In [ ]:
if wandb_api_key:
  wandb.login(key=wandb_api_key)
  print("Successfully logged in to WANDB!")
else:
    print("WANDB key not found in notebook secrets.")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ronchaudhuri29 (ronchaudhuri29-the-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Successfully logged in to WANDB!


# <font color = 'indianred'> **Load Data set**
    


In [ ]:
data_folder = base_folder/'datasets'
df = pd.read_csv(data_folder/'train.csv')
df

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1
...,...,...,...,...,...,...,...,...,...
2024,2024,Please edit your post so it is readable. These...,No legal advice: Do not offer or request legal...,relationships,"I'm not ok with this in anyway, and think you ...",See a lawyer under the guise that you want thi...,"This is just untrue. OP is 13, not an adult -...",Why should I care about all the bicyclists I i...,1
2025,2025,"Yes, and in a right to work state they can eve...",No legal advice: Do not offer or request legal...,legaladvice,Move as much of your assets as you can offshor...,We have great consumer protection laws. There'...,"LPT piratebay, transmission, vpn. Get the musi...","It's not so much that I killed them, it's that...",0
2026,2026,**HD** Streams: |ENG **HD**[ Watch here..PC & ...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,stitopdisca1987.tumblr.com - sex Take girl for...,this girl get sex going to to old http://mrk....,NO ADS | NO ADS | NO ADS\n\nWe show all SOCCER...,[So great! Thanks for sharing.](http://www.che...,1
2027,2027,No. Not when doing so obviously presents a saf...,No legal advice: Do not offer or request legal...,politics,SHE ISNT A BIRTHING CHAMBER BUT EQUALLY THE BA...,"Jail? What are you, ten years old? If they pro...",Who cares about that when I can keep raping in...,send me a private message; I may be able to he...,1


In [ ]:
jigsaw_dataset = Dataset.from_pandas(df)

In [ ]:
selected_columns = {
    "text": jigsaw_dataset["body"],            # comment text
    "rule": jigsaw_dataset["rule"],            # subreddit rule
    "subreddit": jigsaw_dataset["subreddit"],  # subreddit name
    "tag": jigsaw_dataset["rule_violation"],   # label
}
jigsaw_selected_columns = Dataset.from_dict(selected_columns)

In [ ]:
# Set the format to Pandas
jigsaw_selected_columns.set_format(type='pandas')

In [ ]:
df = jigsaw_selected_columns[:]

In [ ]:
df['label'] = df['tag'].astype(int)

In [ ]:
df["text"] = (
    "COMMENT: " + df["text"].astype(str).str.strip() + "\n"
    + "RULE: " + df["rule"].astype(str).str.strip() + "\n"
    + "SUBREDDIT: " + df["subreddit"].astype(str).str.strip()
)

In [ ]:
df['label_text'] = df['label'].map({0: 'complies', 1: 'violates'})

In [ ]:
df_final = df[['text', 'label_text']]

In [ ]:
jigsaw_selected_column_final = Dataset.from_pandas(df_final, preserve_index=False)

# <font color = 'indianred'> **Accessing and Manuplating Splits**</font>



<font color = 'indianred'>*Create futher subdivions of the splits*</font>

In [ ]:
# Split the test set into test and validation sets
test_val_splits = jigsaw_selected_column_final.train_test_split(test_size=0.2, seed=42)
train_split= test_val_splits['train']
test_val_splits = test_val_splits['test'].train_test_split(test_size=0.5, seed=42,)
val_split = test_val_splits['train']
test_split = test_val_splits['test']

<font color = 'indianred'>*small subset for initial experimenttaion*</font>

In [ ]:
data_subset = DatasetDict({"train": train_split, "valid": val_split, "test": test_split})

In [ ]:
data_subset

DatasetDict({
    train: Dataset({
        features: ['text', 'label_text'],
        num_rows: 1623
    })
    valid: Dataset({
        features: ['text', 'label_text'],
        num_rows: 203
    })
    test: Dataset({
        features: ['text', 'label_text'],
        num_rows: 203
    })
})

In [ ]:
data_subset['train'][0]

{'text': "COMMENT: unethical but... make a SS# but state name and address and all perfectly.  If the IRS asks, just say that's the number you got, let the IRS sort if out for you.\nRULE: No legal advice: Do not offer or request legal advice.\nSUBREDDIT: personalfinance",
 'label_text': 'violates'}

# <font color = 'indianred'>**Load pre-trained Tokenizer**</font>

In [ ]:
def free_gpu_memory():
    """
    Frees up GPU memory after CUDA out-of-memory error in Colab.

    This function performs the following steps:
    1. Deletes all PyTorch objects to clear references.
    2. Calls garbage collection to remove unreferenced objects from memory.
    3. Uses torch.cuda.empty_cache() to release cached GPU memory.
    4. Waits for a moment to ensure memory is fully released.
    """
    try:
        # Delete all torch tensors to free up memory
        for obj in list(locals().values()):
            if torch.is_tensor(obj):
                del obj

        # Collect garbage to release any remaining unused memory
        gc.collect()

        # Empty the CUDA cache to release GPU memory
        torch.cuda.empty_cache()

        # Adding a small delay to allow memory to be fully released
        time.sleep(2)

        print("GPU memory has been freed.")
    except Exception as e:
        print(f"Error while freeing GPU memory: {e}")

In [ ]:
free_gpu_memory()

GPU memory has been freed.


In [ ]:
checkpoint = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenizer.eos_token

'<eos>'

In [ ]:
tokenizer.pad_token

'<pad>'

In [ ]:
tokenizer.padding_side

'left'

In [ ]:
from pprint import pprint
pprint(tokenizer.chat_template)

("{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ "
 "raise_exception('System role not supported') }}{% endif %}{% for message in "
 "messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ "
 "raise_exception('Conversation roles must alternate "
 "user/assistant/user/assistant/...') }}{% endif %}{% if (message['role'] == "
 "'assistant') %}{% set role = 'model' %}{% else %}{% set role = "
 "message['role'] %}{% endif %}{{ '<start_of_turn>' + role + '\n"
 "' + message['content'] | trim + '<end_of_turn>\n"
 "' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n"
 "'}}{% endif %}")


In [ ]:
# 2. Test the template works
print("Testing custom chat template...")
test_result = tokenizer.apply_chat_template(
    [{"role": "user", "content": "Test"}, {"role": "assistant", "content": "Response"}],
    return_dict=True,
    return_assistant_tokens_mask=True,
    add_generation_prompt=False
)
print(test_result)
assistant_tokens = sum(test_result.get('assistant_masks', []))
print(f"Assistant tokens detected: {assistant_tokens}")

Testing custom chat template...
{'input_ids': [2, 106, 1645, 108, 2015, 107, 108, 106, 2516, 108, 3943, 107, 108], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'assistant_masks': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
Assistant tokens detected: 0


In [ ]:
# Modified template to only train on the actual content
tokenizer.chat_template = """
{{ bos_token }}
{% if messages[0]['role'] == 'system' %}
  {{ raise_exception('System role not supported') }}
{% endif %}
{% for message in messages %}
  {% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}
    {{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}
  {% endif %}
  {% if message['role'] == 'assistant' %}
<start_of_turn>model
{% generation %}{{ message['content'] | trim }}{% endgeneration %}
<end_of_turn>
  {% else %}
<start_of_turn>{{ message['role'] }}
{{ message['content'] | trim }}
<end_of_turn>
  {% endif %}
{% endfor %}
{% if add_generation_prompt %}
<start_of_turn>model
{% endif %}
"""


In [ ]:
# 2. Test the template works
print("Testing custom chat template...")
test_result = tokenizer.apply_chat_template(
    [{"role": "user", "content": "Test"}, {"role": "assistant", "content": "Response"}],
    return_dict=True,
    return_assistant_tokens_mask=True,
    add_generation_prompt=False
)
print(test_result)
assistant_tokens = sum(test_result.get('assistant_masks', []))
print(f"Assistant tokens detected: {assistant_tokens}")

Testing custom chat template...
{'input_ids': [108, 2, 108, 106, 1645, 108, 2015, 108, 107, 108, 106, 2516, 108, 3943, 107, 108], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'assistant_masks': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]}
Assistant tokens detected: 1


In [ ]:
# Decode individual tokens to see what they are
print("\nToken-by-token breakdown:")
print("Index | Token ID | Token Text | Assistant?")
print("-" * 50)
input_ids = test_result['input_ids']
assistant_masks = test_result['assistant_masks']

for i, (token_id, is_assistant) in enumerate(zip(input_ids, assistant_masks)):
    token_text = tokenizer.decode([token_id])
    status = "ASSISTANT" if is_assistant else "Other"
    print(f"{i:5d} | {token_id:8d} | {repr(token_text):15s} | {status}")

print(f"\nFull text: {repr(tokenizer.decode(input_ids))}")

# Show only assistant tokens
assistant_token_ids = [token_id for token_id, is_assistant in zip(input_ids, assistant_masks) if is_assistant]
assistant_only_text = tokenizer.decode(assistant_token_ids)
print(f"Assistant-only text: {repr(assistant_only_text)}")

# Show the masked (non-assistant) tokens
user_token_ids = [token_id for token_id, is_assistant in zip(input_ids, assistant_masks) if not is_assistant]
user_only_text = tokenizer.decode(user_token_ids)
print(f"User/Other text: {repr(user_only_text)}")


Token-by-token breakdown:
Index | Token ID | Token Text | Assistant?
--------------------------------------------------
    0 |      108 | '\n'            | Other
    1 |        2 | '<bos>'         | Other
    2 |      108 | '\n'            | Other
    3 |      106 | '<start_of_turn>' | Other
    4 |     1645 | 'user'          | Other
    5 |      108 | '\n'            | Other
    6 |     2015 | 'Test'          | Other
    7 |      108 | '\n'            | Other
    8 |      107 | '<end_of_turn>' | Other
    9 |      108 | '\n'            | Other
   10 |      106 | '<start_of_turn>' | Other
   11 |     2516 | 'model'         | Other
   12 |      108 | '\n'            | Other
   13 |     3943 | 'Response'      | ASSISTANT
   14 |      107 | '<end_of_turn>' | Other
   15 |      108 | '\n'            | Other

Full text: '\n<bos>\n<start_of_turn>user\nTest\n<end_of_turn>\n<start_of_turn>model\nResponse<end_of_turn>\n'
Assistant-only text: 'Response'
User/Other text: '\n<bos>\n<start_of_tur

#<font color = 'indianred'> **Create Chat Dataset**



In [ ]:
def format_chat(example):
    instruction = (
        "You are a content policy checker. Analyze the given COMMENT, RULE, "
        "and SUBREDDIT context below. Output exactly one word:\n"
        "- 'complies' if the comment follows the subreddit rules, or\n"
        "- 'violates' if it breaks them.\n\n"
        f"{example['text']}"
    )

    messages = [
        {"role": "user", "content": instruction},
        {"role": "assistant", "content": example["label_text"]}
    ]

    return {"messages": messages}


In [ ]:
data_subset_chat = data_subset.map(format_chat, remove_columns=["text","label_text"])

Map:   0%|          | 0/1623 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [ ]:
data_subset_chat

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 1623
    })
    valid: Dataset({
        features: ['messages'],
        num_rows: 203
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 203
    })
})

In [ ]:
data_subset_chat['train'][0]

{'messages': [{'content': "You are a content policy checker. Analyze the given COMMENT, RULE, and SUBREDDIT context below. Output exactly one word:\n- 'complies' if the comment follows the subreddit rules, or\n- 'violates' if it breaks them.\n\nCOMMENT: unethical but... make a SS# but state name and address and all perfectly.  If the IRS asks, just say that's the number you got, let the IRS sort if out for you.\nRULE: No legal advice: Do not offer or request legal advice.\nSUBREDDIT: personalfinance",
   'role': 'user'},
  {'content': 'violates', 'role': 'assistant'}]}

##  <font color = 'indianred'> **Filter Longer sequences**

In [ ]:
def check_length(example):
    # Find the first user message
    text_to_check = None
    for msg in example.get("messages", []):
        if msg.get("role") == "user":
            text_to_check = msg.get("content", "").strip()
            break

    # Return False if there's no user message or empty text
    if not text_to_check:
        return False

    # Tokenize the text and check length
    encoding = tokenizer.encode(text_to_check)
    return len(encoding) <= 1024


In [ ]:
filtered_data_subset_chat = DatasetDict({
    split: data_subset_chat[split].filter(check_length)
    for split in data_subset_chat.keys()
})

Filter:   0%|          | 0/1623 [00:00<?, ? examples/s]

Filter:   0%|          | 0/203 [00:00<?, ? examples/s]

Filter:   0%|          | 0/203 [00:00<?, ? examples/s]

In [ ]:
filtered_data_subset_chat

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 1623
    })
    valid: Dataset({
        features: ['messages'],
        num_rows: 203
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 203
    })
})

In [ ]:
filtered_data_subset_chat['train'][0]

{'messages': [{'content': "You are a content policy checker. Analyze the given COMMENT, RULE, and SUBREDDIT context below. Output exactly one word:\n- 'complies' if the comment follows the subreddit rules, or\n- 'violates' if it breaks them.\n\nCOMMENT: unethical but... make a SS# but state name and address and all perfectly.  If the IRS asks, just say that's the number you got, let the IRS sort if out for you.\nRULE: No legal advice: Do not offer or request legal advice.\nSUBREDDIT: personalfinance",
   'role': 'user'},
  {'content': 'violates', 'role': 'assistant'}]}

##  <font color = 'indianred'> **Push Dataset to Hub**

In [ ]:
filtered_data_subset_chat.push_to_hub(
    "Ron00769/jigsaw_binary_subset_chat",
    private=False
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########|  258kB /  258kB            

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########| 34.0kB / 34.0kB            

README.md:   0%|          | 0.00/540 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Ron00769/jigsaw_binary_subset_chat/commit/01485c6231dd43778869b041351f3102bd7be278', commit_message='Upload dataset', commit_description='', oid='01485c6231dd43778869b041351f3102bd7be278', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Ron00769/jigsaw_binary_subset_chat', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Ron00769/jigsaw_binary_subset_chat'), pr_revision=None, pr_num=None)

#  <font color = 'indianred'> **Model Training**

##  <font color = 'indianred'> **Download pre-trained model**

In [ ]:
def get_appropriate_dtype():
    if torch.cuda.is_available() and torch.cuda.get_device_capability(0) >= (8, 0):
        return torch.bfloat16
    return torch.float16

In [ ]:
torch_data_type = get_appropriate_dtype()
torch_data_type

torch.bfloat16

In [ ]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_use_double_quant=True,
  bnb_4bit_compute_dtype=torch_data_type,
  bnb_4bit_quant_storage=torch_data_type,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                             quantization_config=bnb_config,
                                             torch_dtype=torch_data_type,
                                             trust_remote_code=True,)


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

##  <font color = 'indianred'> **PEFT Setup**

In [ ]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedfor

In [ ]:
def find_linear_layers(model):
    """
    Extracts the unique names of Linear layers from a model.

    Args:
    model (nn.Module): The model from which to extract Linear layer names.

    Returns:
    list: A list of unique names of Linear layers.
    """
    # Convert the model's modules to string
    model_modules = str(model.modules)
    # Pattern to extract names of Linear layers
    pattern = r'\((\w+)\): Linear'
    # Find all occurrences of the pattern
    linear_layer_names = re.findall(pattern, model_modules)
    print(linear_layer_names)
    # Get unique names using a set, then convert back to list
    target_modules = list(set(linear_layer_names))
    return target_modules

In [ ]:
find_linear_layers(model)

['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'lm_head']


['k_proj',
 'o_proj',
 'v_proj',
 'lm_head',
 'down_proj',
 'gate_proj',
 'up_proj',
 'q_proj']

In [ ]:
TaskType.CAUSAL_LM

<TaskType.CAUSAL_LM: 'CAUSAL_LM'>

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=128,
    lora_alpha=256,
    lora_dropout=0.01,
    target_modules = ['v_proj',  'q_proj',  'up_proj', 'o_proj', 'down_proj', 'gate_proj','k_proj'])



## <font color = 'indianred'> **Training Arguments**</font>







In [ ]:
# Define the directory where model checkpoints will be saved
model_folder = base_folder/'models/jigsaw/decoder_with_instruction_language_head'

# Create the directory if it doesn't exist
model_folder.mkdir(exist_ok=True, parents=True)
run_name= 'jigsaw_inst_exp_lmh_gemma_inst'

use_fp16 = torch_data_type == torch.float16
use_bf16 = torch_data_type == torch.bfloat16

# Configure training parameters
training_args = SFTConfig(
    seed = 42,
    dataset_text_field="text",
    max_length = 1024,
    packing = False,
    assistant_only_loss=True,

    # Training-specific configurations
    num_train_epochs=2,  # Total number of training epochs
    per_device_train_batch_size=8, # Number of samples per training batch for each device
    per_device_eval_batch_size=8,  # Number of samples per evaluation batch for each device
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant":False},
    # torch_empty_cache_steps=5,
    weight_decay=0.0,  # Apply L2 regularization to prevent overfitting
    learning_rate=1e-5,  # Step size for the optimizer during training
    optim='adamw_torch',  # Optimizer,

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder),  # Directory to save model checkpoints
    eval_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=20,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=20,  # Save a model checkpoint every 10 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=2,  # Retain only the best and the most recent model checkpoints
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="eval_loss",
    greater_is_better=False,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations (commented out in this example)
    logging_strategy='steps',
    logging_steps=20,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name= run_name,  # Experiment name for Weights & Biases

    # Precision settings determined based on GPU capability
    fp16=use_fp16 ,  # Set True if torch_data_type is torch.float16
    bf16=use_bf16,  # Set True if torch_data_type is torch.bfloat16
    tf32=False,  # Disable tf32 unless you want to use Ampere specific optimization
)


In [ ]:
# If gradient checkpointing is enabled, configure relevant settings
if training_args.gradient_checkpointing:
    model.config.use_cache = False  # Disable caching for compatibility



##  <font color = 'indianred'> **Initialize Trainer**</font>



In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=filtered_data_subset_chat['train'],
    eval_dataset=filtered_data_subset_chat['valid'],
    peft_config=peft_config,
    processing_class=tokenizer,
)

Tokenizing train dataset:   0%|          | 0/1623 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1623 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/203 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/203 [00:00<?, ? examples/s]

In [ ]:
dataloader = trainer.get_train_dataloader()
batch = next(iter(dataloader))

In [ ]:
batch

{'input_ids': tensor([[108,   2, 108,  ...,   0,   0,   0],
         [108,   2, 108,  ...,   0,   0,   0],
         [108,   2, 108,  ...,   0,   0,   0],
         ...,
         [108,   2, 108,  ...,   0,   0,   0],
         [108,   2, 108,  ...,   0,   0,   0],
         [108,   2, 108,  ...,   0,   0,   0]], device='cuda:0'),
 'labels': tensor([[-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         ...,
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100],
         [-100, -100, -100,  ..., -100, -100, -100]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [ ]:
print(batch['input_ids'][0][0:5])
print(tokenizer.decode(batch['input_ids'][0][0:5]))
print(batch['labels'][0][0:5])

tensor([ 108,    2,  108,  106, 1645], device='cuda:0')

<bos>
<start_of_turn>user
tensor([-100, -100, -100, -100, -100], device='cuda:0')


In [ ]:
print(len(batch['input_ids'][0]))
print(len(batch['labels'][0]))

228
228


In [ ]:
print(batch['input_ids'][0][0:5])
print(tokenizer.decode(batch['input_ids'][0][0:5]))
print(batch['labels'][0][0:5])

tensor([ 108,    2,  108,  106, 1645], device='cuda:0')

<bos>
<start_of_turn>user
tensor([-100, -100, -100, -100, -100], device='cuda:0')


In [ ]:
print(f"\nINPUTS")
print(f"{'-'*80}")
print(batch['input_ids'][0][106:120])
print(f"\nLABELS")
print(f"{'-'*80}")
print(batch['labels'][0][106:120])
print(f"\nTokens")
print(f"{'-'*80}")
print(tokenizer.decode(batch['input_ids'][0][106:120]))


INPUTS
--------------------------------------------------------------------------------
tensor([ 72364, 109564,    108,    107,    108,    106,   2516,    108,  49515,
          1204,    107,    108,      0,      0], device='cuda:0')

LABELS
--------------------------------------------------------------------------------
tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 49515,  1204,
         -100,  -100,  -100,  -100], device='cuda:0')

Tokens
--------------------------------------------------------------------------------
 Showerthoughts
<end_of_turn>
<start_of_turn>model
violates<end_of_turn>
<pad><pad>


In [ ]:
def verify_loss_masking(tokenizer, trainer, num_samples=3):
    """
    Verify which tokens contribute to loss (labels != -100)
    for a few samples from the training dataloader.
    """
    dataloader = trainer.get_train_dataloader()
    batch = next(iter(dataloader))

    for i in range(min(num_samples, len(batch["input_ids"]))):
        input_ids = batch["input_ids"][i]
        labels = batch["labels"][i]

        print(f"\n{'='*80}")
        print(f"Sample {i+1}")
        print(f"{'='*80}")

        # Decode full sequence for reference
        full_text = tokenizer.decode(input_ids, skip_special_tokens=False)
        pprint(f"\nFull text:\n{full_text}")

        # Identify tokens used for loss
        loss_token_indices = (labels != -100).nonzero(as_tuple=True)[0]

        if len(loss_token_indices) == 0:
            print("All tokens masked — no loss will be calculated.")
            continue

        print(f"\nTokens contributing to loss ({len(loss_token_indices)} total):")
        print(f"{'-'*80}")
        print(f"{'Index':<8} {'Token ID':<10} {'Token Text'}")
        print(f"{'-'*80}")

        for idx in loss_token_indices.tolist():
            token_id = input_ids[idx].item()
            token_text = tokenizer.decode([token_id], skip_special_tokens=False)
            print(f"{idx:<8} {token_id:<10} {repr(token_text)}")

        print(f"{'-'*80}")
        print(f"Percentage of tokens used for loss: {len(loss_token_indices)/len(labels)*100:.2f}%")

In [ ]:
# Call this after creating your trainer
verify_loss_masking(tokenizer, trainer, num_samples=2)



Sample 1
('\n'
 'Full text:\n'
 '\n'
 '<bos>\n'
 '<start_of_turn>user\n'
 'You are a content policy checker. Analyze the given COMMENT, RULE, and '
 'SUBREDDIT context below. Output exactly one word:\n'
 "- 'complies' if the comment follows the subreddit rules, or\n"
 "- 'violates' if it breaks them.\n"
 '\n'
 'COMMENT: naughty holes wait you there http://by.ix-cafe.com/4j3pu\n'
 'RULE: No Advertising: Spam, referral links, unsolicited advertising, and '
 'promotional content are not allowed.\n'
 'SUBREDDIT: Showerthoughts\n'
 '<end_of_turn>\n'
 '<start_of_turn>model\n'
 'violates<end_of_turn>\n'
 '<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

## <font color = 'indianred'> **Setup WandB**</font>

In [ ]:
%env WANDB_PROJECT = jigsaw_inst_language_head2025

env: WANDB_PROJECT=jigsaw_inst_language_head2025


##  <font color = 'indianred'> **Training**

In [ ]:
try:
    # Your code that may cause a CUDA out-of-memory error
    # Example: trainer.train() or other GPU intensive operations
    # lora_model.config.use_cache = False
    trainer.train()
except RuntimeError as e:
    if 'CUDA out of memory' in str(e):
        print("CUDA out of memory error detected. Freeing GPU memory.")
        free_gpu_memory()
        # Optionally, you can retry the operation here after freeing up memory
        # Example retry:
        # trainer.train()
    else:
        raise e


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1}.


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
20,0.659700,0.342797,2.076317,45591.000000,0.766026
40,0.360100,0.326619,2.045369,91228.000000,0.808494
60,0.283300,0.329332,1.955427,137564.000000,0.826923
80,0.258100,0.297278,1.956674,183266.000000,0.858173
100,0.244600,0.275228,1.957899,229401.000000,0.877404
120,0.204500,0.264178,1.953300,273874.000000,0.879808
140,0.206400,0.255462,1.957341,320139.000000,0.882212
160,0.186200,0.287796,1.956602,365895.000000,0.877404
180,0.179200,0.270112,1.958214,410590.000000,0.867788
200,0.172200,0.265974,1.960398,457521.000000,0.877404


##  <font color = 'indianred'> **6.6 Push best checkpoint to Hub**

In [ ]:
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]

In [ ]:
best_model_checkpoint_step

'140'

In [ ]:
checkpoint = str(model_folder/f'checkpoint-{best_model_checkpoint_step}')
checkpoint

'/content/drive/MyDrive/data/models/jigsaw/decoder_with_instruction_language_head/checkpoint-140'

In [ ]:
# Step 1: Create the repository
repo_id="Ron00769/jigsaw_binary_instr_lm_head"
create_repo(
    repo_id=repo_id,
    repo_type="model",
    private=False,  # Set to True if you want it private
    exist_ok=True   # Won't error if repo already exists
)

# Step 2: Upload the folder
api = HfApi()
api.upload_folder(
    folder_path=checkpoint,
    repo_id=repo_id,
    repo_type="model",
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...kpoint-140/tokenizer.json: 100%|##########| 34.4MB / 34.4MB            

  ...point-140/tokenizer.model: 100%|##########| 4.24MB / 4.24MB            

  ...eckpoint-140/optimizer.pt:   0%|          |  569kB / 1.33GB            

  ...adapter_model.safetensors:   0%|          | 43.0kB /  665MB            

  ...eckpoint-140/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

  ...ckpoint-140/rng_state.pth:  77%|#######7  | 11.3kB / 14.6kB            

  ...int-140/training_args.bin:   6%|6         |   400B / 6.42kB            

CommitInfo(commit_url='https://huggingface.co/Ron00769/jigsaw_binary_instr_lm_head/commit/57b3f7be6faa47c3dac5095ebcc7a8ee1c850040', commit_message='Upload folder using huggingface_hub', commit_description='', oid='57b3f7be6faa47c3dac5095ebcc7a8ee1c850040', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ron00769/jigsaw_binary_instr_lm_head', endpoint='https://huggingface.co', repo_type='model', repo_id='Ron00769/jigsaw_binary_instr_lm_head'), pr_revision=None, pr_num=None)

In [ ]:
wandb.finish()

eval/entropy,█▆▁▁▁▁▁▁▁▁
eval/loss,█▇▇▄▃▂▁▄▂▂
eval/mean_token_accuracy,▁▄▅▇████▇█
eval/num_tokens,▁▂▃▃▄▅▆▆▇█
eval/runtime,▁██▄▁▇▃▄▂▁
eval/samples_per_second,█▁▁▅█▂▆▅▇█
eval/steps_per_second,█▁▁▅█▂▆▅▇█
train/entropy,▃█▄▁▃▂▂▂▁▃
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
+5,...
